# Работа с базами данных

## Postgres

### Установка pcycopg

In [24]:
%pip install psycopg2-binary

  Obtaining dependency information for psycopg2-binary from https://files.pythonhosted.org/packages/25/1f/7ae31759142999a8d06b3e250c1346c4abcdcada8fa884376775dc1de686/psycopg2_binary-2.9.9-cp311-cp311-win_amd64.whl.metadata
  Using cached psycopg2_binary-2.9.9-cp311-cp311-win_amd64.whl.metadata (4.6 kB)
Using cached psycopg2_binary-2.9.9-cp311-cp311-win_amd64.whl (1.2 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Отказано в доступе: 'C:\\Users\\Daily\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\psycopg2\\_psycopg.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\Daily\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


### Подключение к базе данных

In [26]:
from psycopg2 import OperationalError, connect
from psycopg2.extensions import connection

def create_connection(db_name:str, db_user:str, db_password:str, db_host:str, db_port:int)->connection:
    connection = None
    try:
        connection = connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
        )
        print("Connection to PostgreSQL DB successful")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    return connection

In [27]:
connection = create_connection( "demo", "postgres", "password", "127.0.0.1", "5432" )

Connection to PostgreSQL DB successful


### Создание таблиц

In [28]:
def execute_query(con: connection, query:str)->None:
    con.autocommit = True
    cursor = con.cursor()
    try:
        cursor.execute(query)
        print("Query executed successfully")
        cursor.close()  # закрываем курсор
    except OperationalError as e:
        cursor.close()  # закрываем курсор
        print(f"The error '{e}' occurred")

In [29]:
create_users_table = """
CREATE TABLE IF NOT EXISTS users (
  id SERIAL PRIMARY KEY,
  name TEXT NOT NULL, 
  age INTEGER,
  gender TEXT,
  nationality TEXT
)
"""

In [30]:
execute_query(connection, create_users_table)
connection.close()    # закрываем подключение

Query executed successfully


### Вставка данных

In [31]:
users = [
    ("James", 25, "male", "USA"),
    ("Leila", 32, "female", "France"),
    ("Brigitte", 35, "female", "England"),
    ("Mike", 40, "male", "Denmark"),
    ("Elizabeth", 21, "female", "Canada"),
]

user_records = ", ".join(["%s"] * len(users))

insert_query = f"INSERT INTO users (name, age, gender, nationality) VALUES {user_records}"

connection = create_connection( "demo", "postgres", "password", "127.0.0.1", "5432" )
connection.autocommit = True
cursor = connection.cursor()
cursor.execute(insert_query, users)
cursor.close()
connection.close()

Connection to PostgreSQL DB successful


### Множественная вставка строк

In [32]:
users = [
    ("James", 25, "male", "USA"),
    ("Leila", 32, "female", "France"),
    ("Brigitte", 35, "female", "England"),
    ("Mike", 40, "male", "Denmark"),
    ("Elizabeth", 21, "female", "Canada"),
]

insert_query = "INSERT INTO users (name, age, gender, nationality) VALUES (%s,%s,%s,%s);"

connection = create_connection( "demo", "postgres", "password", "127.0.0.1", "5432" )
connection.autocommit = True
cursor = connection.cursor()
cursor.executemany(insert_query, users)
cursor.close()
connection.close()

Connection to PostgreSQL DB successful


### Извлечение данных из записей

In [33]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        cursor.close()
        return result
    except OperationalError as e:
        cursor.close()
        print(f"The error '{e}' occurred")

select_users = "SELECT * FROM users"

connection = create_connection( "demo", "postgres", "password", "127.0.0.1", "5432" )
users = execute_read_query(connection, select_users)
connection.close()

for user in users:
    print(user)

Connection to PostgreSQL DB successful
(1, 'James', 25, 'male', 'USA')
(2, 'Leila', 32, 'female', 'France')
(3, 'Brigitte', 35, 'female', 'England')
(4, 'Mike', 40, 'male', 'Denmark')
(5, 'Elizabeth', 21, 'female', 'Canada')
(6, 'James', 25, 'male', 'USA')
(7, 'Leila', 32, 'female', 'France')
(8, 'Brigitte', 35, 'female', 'England')
(9, 'Mike', 40, 'male', 'Denmark')
(10, 'Elizabeth', 21, 'female', 'Canada')


### Обновление записей в таблице

In [34]:
update_user_nationality = """
UPDATE
  users
SET
  nationality = 'russian'
WHERE
  id = 2
"""

connection = create_connection( "demo", "postgres", "password", "127.0.0.1", "5432" )
execute_query(connection,  update_user_nationality)
connection.close()

Connection to PostgreSQL DB successful
Query executed successfully


### Удаление записей в т

In [35]:
delete_user = "DELETE FROM users WHERE id = 5"

connection = create_connection( "demo", "postgres", "password", "127.0.0.1", "5432" )
execute_query(connection, delete_user)
connection.close()

Connection to PostgreSQL DB successful
Query executed successfully


### Pandas и Psycopg2

In [36]:
import pandas as pd

In [37]:
query = '''
select fare_conditions , sum(tf.amount) from tickets t left join ticket_flights tf on tf.ticket_no = t.ticket_no group by fare_conditions ;
'''

In [38]:
connection = create_connection( "demo", "postgres", "password", "127.0.0.1", "5432" )
data_frame = pd.read_sql(query,connection)
connection.close()

Connection to PostgreSQL DB successful


C:\Users\Daily\AppData\Local\Temp\ipykernel_868\4238980040.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_frame = pd.read_sql(query,connection)


In [39]:
data_frame

,fare_conditions,sum
0,Business,5.505180e+09
1,Comfort,5.661169e+08
2,Economy,1.469568e+10


### Pandas и Sqlalchemy

In [41]:
%pip install sqlalchemy

  Obtaining dependency information for sqlalchemy from https://files.pythonhosted.org/packages/67/e7/7c77fd5290646f929b499992607cf1bc940573098a593080fcc8f7e13a08/SQLAlchemy-2.0.23-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.2.0 from https://files.pythonhosted.org/packages/b7/f4/6a90020cd2d93349b442bfcb657d0dc91eee65491600b2cb1d388bc98e6b/typing_extensions-4.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for greenlet!=0.4.17 from https://files.pythonhosted.org/packages/82/46/09045a85d4df1b63433c557663ffac664ce7797f21691a16ddebca4f4b56/greenlet-3.0.2-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
    --------------------------------------- 0.0/2.1 MB 435.7 kB/s eta 0:00:05
    --------------------------------------- 0.0/2.1 MB 4


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\Daily\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [42]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.sql import text

sql_query = '''
select fare_conditions , sum(tf.amount) from tickets t left join ticket_flights tf on tf.ticket_no = t.ticket_no group by fare_conditions ;
'''

engine = create_engine(
    "{dialect}+{driver}://{username}:{password}@{host}:{port}/{database}".format(
        dialect="postgresql",
        driver="psycopg2",
        username="postgres",
        password="password",
        host="localhost",
        port=5432,
        database="demo"
    )
)

with engine.connect() as db_conn:
    data_frame = pd.read_sql(sql=text(sql_query), con=db_conn)
data_frame

,fare_conditions,sum
0,Business,5.505180e+09
1,Comfort,5.661169e+08
2,Economy,1.469568e+10


## Clickhouse

In [43]:
%pip install clickhouse-connect

Note: you may need to restart the kernel to use updated packages.Collecting clickhouse-connect
  Obtaining dependency information for clickhouse-connect from https://files.pythonhosted.org/packages/d2/29/b2e4a1f33d0b369de34164f8287920d877928e1f997a064a62833adeafc0/clickhouse_connect-0.6.23-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for zstandard from https://files.pythonhosted.org/packages/8a/bc/878a5b8f413d5fe902842fde08ecf317d3979c9728135034a519c409ce3f/zstandard-0.22.0-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/99.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/99.8 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/99.8 kB ? eta -:--:--
     ----------- -------------------------- 30.7/99.8 kB 259.2 kB/s eta 0:00:01
     ----------- -------------------------- 30.7/99.8 kB 259.2 kB/s eta 0:00:01
     --------------- ---------------------- 41.0/99.8 kB 178.6 kB/s eta 0:00:01
    


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\Daily\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


### Подключение к базе данных

In [1]:
import clickhouse_connect

client = clickhouse_connect.get_client(host='localhost', username='default', password='')

### Создание таблиц

In [2]:
client.command('CREATE TABLE new_table (key UInt32, value String, metric Float64) ENGINE MergeTree ORDER BY key')

### Вставка данных

In [3]:
row1 = [1000, 'String Value 1000', 5.233]
row2 = [2000, 'String Value 2000', -107.04]
data = [row1, row2]
client.insert('new_table', data, column_names=['key', 'value', 'metric'])

### Извлечение данных из таблиц

In [4]:
result = client.query('SELECT max(key), avg(metric) FROM new_table')
result.result_rows

[(2000, -50.9035)]

### Pandas и clickhouse

In [5]:
df_stream = client.query_df_stream('SELECT * FROM new_table')
column_names = df_stream.source.column_names
with df_stream:
    for df in df_stream:
        print(df)

    key              value   metric
0  1000  String Value 1000    5.233
1  2000  String Value 2000 -107.040
